In [3]:
import pandas as pd

In [1]:
# load df from csv
df = pd.read_csv('dataset_phishing.csv')

NameError: name 'pd' is not defined

In [8]:
# taken from here, we want to remove them because we can't use them in our model
# https://data.mendeley.com/datasets/c2gw7fy2j4/3/files/562f107e-e96e-47f9-8635-a326b1d5fae4
external_features = [
    "domain_registration_length",
    # "domain_registration_length1",
    "whois_registered_domain",
    "web_traffic",
    "domain_age",
    # "global_rank",
    "google_index",
    "dns_record",
    "page_rank",
]

In [9]:
df = df.drop(columns=external_features)

In [10]:
df

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,0,0,0.000000,0,0,0,0,1,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,0,0,0,100.000000,0,0,0,1,0,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,0,0,0,100.000000,0,0,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,0,0,0,62.500000,0,0,0,1,0,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,0,0,0.000000,0,0,0,0,1,legitimate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,45,17,0,2,0,0,0,0,0,...,0,0,0,0.000000,0,0,0,0,0,legitimate
11426,http://www.budgetbots.com/server.php/Server%20...,84,18,0,5,0,1,1,0,0,...,0,0,0,0.000000,0,0,0,1,0,phishing
11427,https://www.facebook.com/Interactive-Televisio...,105,16,1,2,6,0,1,0,0,...,0,0,0,80.000000,0,0,0,0,0,legitimate
11428,http://www.mypublicdomainpictures.com/,38,30,0,2,0,0,0,0,0,...,0,0,0,0.000000,0,0,0,1,0,legitimate


In [21]:
df['status'] = df['status'].map({"legitimate": 0, "phishing": 1})

In [22]:
corr = df.corr()

/var/folders/02/q__yg7mx1cn27p_3747mry900000gn/T/ipykernel_83048/361440739.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


In [23]:
corr["status"].sort_values(ascending=False)

status                  1.000000
ratio_digits_url        0.356395
domain_in_title         0.342807
phish_hints             0.335393
ip                      0.321698
                          ...   
ratio_nullHyperlinks         NaN
ratio_intRedirection         NaN
ratio_intErrors              NaN
submit_email                 NaN
sfh                          NaN
Name: status, Length: 81, dtype: float64

In [82]:
low_corr_features = corr["status"].sort_values(ascending=False).index[abs(corr["status"]) < 0.05]

In [83]:
low_corr_features

Index(['ratio_digits_host', 'shortest_word_host', 'prefix_suffix',
       'tld_in_subdomain', 'nb_dots', 'avg_word_path', 'avg_word_host',
       'longest_word_host', 'https_token', 'ratio_extHyperlinks',
       'shortest_word_path', 'nb_dslash', 'brand_in_subdomain',
       'nb_external_redirection', 'nb_hyphens', 'ratio_extMedia',
       'ratio_intHyperlinks', 'nb_or', 'ratio_nullHyperlinks'],
      dtype='object')

In [93]:
from sklearn.model_selection import train_test_split

X = df.drop(["status", "url"], axis=1)
# remove features with small correselation with status
# X = X.drop(low_corr_features, axis=1)
# X.columns = [f'f{i}' for i in range(X.shape[1])]
y = df["status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [85]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

# Hyperparameter grids
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
}

adb_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1],
}

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

# Grid search
# rf_search = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=3, scoring='accuracy')
# adb_search = GridSearchCV(AdaBoostClassifier(random_state=42), adb_param_grid, cv=3, scoring='accuracy')
xgb_search = GridSearchCV(XGBClassifier(eval_metric='logloss', random_state=42),
                          xgb_param_grid, cv=3, scoring='accuracy')

# Fit models
# rf_search.fit(X_train, y_train)
# adb_search.fit(X_train, y_train)
xgb_search.fit(X_train, y_train)

# Best models
# best_rf = rf_search.best_estimator_
# best_adb = adb_search.best_estimator_
best_xgb = xgb_search.best_estimator_

# print("Best RF params:", rf_search.best_params_)
# print("Best AdaBoost params:", adb_search.best_params_)
print("Best XGBoost params:", xgb_search.best_params_)


Best XGBoost params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


In [94]:
from sklearn.feature_selection import RFE

rfe = RFE(XGBClassifier(eval_metric='logloss', random_state=42), n_features_to_select=30)
rfe.fit(X_train, y_train)

selected_features = X.columns[rfe.support_]
X_reduced = X[selected_features]

In [95]:
selected_features

Index(['length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm',
       'nb_eq', 'nb_underscore', 'nb_slash', 'nb_www', 'ratio_digits_url',
       'ratio_digits_host', 'prefix_suffix', 'shortening_service',
       'length_words_raw', 'shortest_word_path', 'longest_words_raw',
       'longest_word_path', 'phish_hints', 'domain_in_brand', 'suspecious_tld',
       'nb_hyperlinks', 'ratio_extHyperlinks', 'login_form',
       'external_favicon', 'links_in_tags', 'safe_anchor', 'empty_title',
       'domain_in_title', 'domain_with_copyright'],
      dtype='object')

In [90]:
from sklearn.metrics import classification_report

y_pred = rfe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1157
           1       0.95      0.94      0.94      1129

    accuracy                           0.95      2286
   macro avg       0.95      0.95      0.95      2286
weighted avg       0.95      0.95      0.95      2286



In [58]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Define stacking classifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', best_rf),
        ('adb', best_adb),
        ('xgb', best_xgb)
    ],
    final_estimator=LogisticRegression(),
    cv=5
)

# Fit stacking model
stacking_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = stacking_model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1157
           1       0.96      0.94      0.95      1129

    accuracy                           0.95      2286
   macro avg       0.95      0.95      0.95      2286
weighted avg       0.95      0.95      0.95      2286



In [51]:
y_pred = best_xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1157
           1       0.96      0.95      0.95      1129

    accuracy                           0.95      2286
   macro avg       0.95      0.95      0.95      2286
weighted avg       0.95      0.95      0.95      2286



In [59]:
import joblib

# Save the stacking model
joblib.dump(stacking_model, 'stacking_model.pkl')

# Load the stacking model later
# loaded_model = joblib.load('stacking_model.pkl')

['stacking_model.pkl']

In [52]:
import joblib

# Save the stacking model
joblib.dump(best_xgb, 'xgb_model.pkl')

# Load the stacking model later
# loaded_model = joblib.load('stacking_model.pkl')

['xgb_model.pkl']

In [55]:
from onnxconverter_common import FloatTensorType
from skl2onnx import convert_sklearn

convert_sklearn(
    best_xgb,
    "pipeline_xgboost",
    [("input", FloatTensorType([None, 2]))],
    target_opset={"": 12, "ai.onnx.ml": 2},
)

RuntimeError: Unable to interpret 'nb_hyperlinks', feature names should follow pattern 'f%d'.

In [65]:
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = skl2onnx.convert_sklearn(best_xgb, initial_types=initial_type, target_opset={"": 12, "ai.onnx.ml": 3})

# Save model
with open("xgb.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


In [26]:
urls = pd.read_csv("data/dataset.csv")

/var/folders/02/q__yg7mx1cn27p_3747mry900000gn/T/ipykernel_7954/2292731433.py:1: DtypeWarning: Columns (1,33) have mixed types. Specify dtype option on import or set low_memory=False.
  urls = pd.read_csv("data/dataset.csv")


In [27]:
urls.shape

(223921, 34)

In [28]:
urls = urls[urls["length_hostname"] != "invalid"]

In [29]:
urls.shape

(97813, 34)

In [30]:
urls = urls[urls["status"] != "invalid"]

In [31]:
urls.shape

(89315, 34)

In [34]:
urls[urls["status"] == "phishing"].shape

(49103, 34)

In [39]:
# print all values of status column
urls["status"].unique()

array(['phishing', '0.0', '1.0', 'legitimate'], dtype=object)

In [47]:
legitimate_urls = urls[urls["status"] == "legitimate"]
phishing_urls = urls[urls["status"] == "phishing"]

In [50]:
# save concatenated dataframe to csv
concatenated_urls = pd.concat([legitimate_urls, phishing_urls])
concatenated_urls.to_csv("data/urls_with_features.csv", index=False)